# Week 3: Basic Document Classification (Part 2)

## Preliminaries 

In [4]:
#necessary library imports and setup introduced previously

import sys
#sys.path.append(r'T:\Departments\Informatics\LanguageEngineering') 
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'\Users\J\Desktop\code\sussex\nlp\labs\resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.tokenize import word_tokenize
from nltk.classify.api import ClassifierI

from sussex_nltk.corpus_readers import ReutersCorpusReader

Sussex NLTK root directory is \Users\J\Desktop\code\sussex\nlp\labs\resources


## Overview 
In labs this week (and next), the focus is on the application of sentiment analysis. You have started using the **Amazon review corpus** and exploring various techniques that can be used to classify the sentiment of Amazon book reviews as either positive or negative. 

Last time, you developed your own **Word List** classifiers.  This time you will be developing a **Naïve Bayes** classifiers.

## Naïve Bayes classifiers

We will introduce Naïve Bayes classifiers through a very simple example dataset involving documents that are either about the weather or football. The classifier will be trained to distinguish these two topics from one another.

There are, therefore, two classes `weather` and `football`. The classifier's job is to determine whether a document that it is given is in the class `weather` or in the class `football`.

We give the classifier examples of documents in the `weather` class, and examples of documents in the `football` class. For now, to keep things simple, our so-called documents will be very short phrases.

Run the following cell to set up some training data.

In [3]:
# Some test data to get us started. It is deliberately VERY simple. 

weather_sents_train = [
    "today it is raining",
    "looking cloudy today",
    "it is nice weather",
]

football_sents_train = [
    "city looking good",
    "advantage united",
]

### Assumptions
One of the simplying assumptions that is made with Naïve Bayes classification is that each document is taken to be a so-called **bag of words**. What this means is that the word order is ignored. For example, the document phrase "city looking good" will generate the same *bag* of words as "good looking city" and "good city looking"

The second assumption that we make is that we use the **multi-variate Bernouilli event model**.   This means that we do not taking into account the number of times a word appears in a document.   Each variable (i.e., word) has two possible values -- `True` if it occurs and `False` if it does not.  Note that there are **multinomial** variants of Naïve Bayes where the number of times a words appears in a document is taken into account, but we will not consider that case here.

Given these assumptions, we will represent each training document as a pair consisting of a dictionary that maps each word that appears in the document to `True`, and a string denoting the class of the document. 

### Exercise 1.1
In the cell below, write code that achieves this. Create three lists:
- `weather_data_train`: a dictionary containing the data for documents in the class `weather`;
- `football_data_train`: a dictionary containing the data for documents in the class `football`;
- `train_data`: which is simply `weather_data_train + football_data_train`

For example, `weather_data_train` should look like:

<img src='files/traindataex.png'>

Hint: this can be done with nested list/dictionary comprehensions.

In [131]:
def tokenDict(words):
    return {word: True for word in words}

def listTokenDicts(train, name):
    return [(tokenDict(sentence.split()), name) for sentence in train]

weather_data_train = listTokenDicts(weather_sents_train, 'weather')
football_data_train = listTokenDicts(weather_sents_train, 'football')
train_data = weather_data_train + football_data_train
print(weather_data_train)

[({'today': True, 'it': True, 'is': True, 'raining': True}, 'weather'), ({'looking': True, 'cloudy': True, 'today': True}, 'weather'), ({'it': True, 'is': True, 'nice': True, 'weather': True}, 'weather')]


### How a Naïve Bayes classifier works

We will now look at how a NB work, using our `weather` versus `football` classification task.

In order to classify a document, $d$, we need to determine which of these probabilities is greatest:

$$P(\,\mbox{weather}\,|\,d) \qquad\qquad \mbox{versus} \qquad\qquad P(\,\mbox{football}\,|\,d)$$

$d$ could, for example, be the string "today is looking cloudy", which would give us:

$$P(\,\mbox{weather}\,|\,\mbox{"today is looking cloudy"}) \qquad\qquad \mbox{versus} \qquad\qquad P(\,\mbox{football}\,|\,\mbox{"today is looking cloudy"})$$

The idea is that if the term on the left is higher then the document is in category `weather`, and if the term on the right is higher then the document is in category `football`.

$P(X|Y)$ means the probability of $X$ given $Y$. So, $P(\,\mbox{weather}\,|\,d)$ means the probability, given a document $d$, of $d$ being of class `weather`.

We are going to use something called Bayes' rule which states that:

$$P(X|Y) = \frac{P(Y|X)\cdot P(X)}{P(Y)}$$

Applying Bayes' rule to our problem leads to the following comparision:

$$\frac{P(\,d\,|\,\mbox{weather}\,)\cdot P(\,\mbox{weather}\,)}{p(d)} \qquad\qquad \mbox{versus} \qquad\qquad \frac{P(\,d\,|\,\mbox{football}\,)\cdot P(\,\mbox{football}\,)}{p(d)}$$

Since both sides are being divided by the same thing, we only need to make the following comparision:

$$P(\,d\,|\,\mbox{weather}\,)\cdot P(\,\mbox{weather}\,) \qquad\qquad \mbox{versus} \qquad\qquad P(\,d\,|\,\mbox{football}\,)\cdot P(\,\mbox{football}\,)$$

Let's just look at what each of these probabilities mean?

1. $P(\,d\,|\,\mbox{weather}\,)$: this is the probability of a document in the `weather` category being the document $d$

2. $P(\,d\,|\,\mbox{football}\,)$: this is the probability of a document in the `football` category being the document $d$

3. $P(\,\mbox{weather}\,)$: this is the probability of a randomly selected document being of category `weather`.

4. $P(\,\mbox{football}\,)$: this is the probability of a randomly selected document being of category `football`.

How are we going to obtain these probabilities? 


### Class priors

We have established that we need to know $P(\,\mbox{weather}\,)$ and $P(\,\mbox{football}\,)$. These are called the class priors. Let's see how we can obtain (estimated) values for these probabilities from our training data.

The classifier has seen three documents of class `weather` and two documents of class `football`.

From this it learns that `weather` documents are slightly more common that `football` documents, and it therefore has a slight bias towards saying a document is a `weather` document.

To be more precise, the classifier has learned that:
- The probability that a document is in class `weather` is $3/5$.  
We say $P(\mbox{weather})=3/5=0.6$.

- The probability that a document is in class `football` is $2/5$.  
We say $P(\mbox{football})=2/5=0.4$.

In general, if the training data contained $n_1$ documents of class `weather` and $n_2$ documents of class `football`, then 

$$P(\mbox{weather})=\frac{n_1}{n_1+n_2} \qquad \mbox{and} \qquad P(\mbox{football})=\frac{n_2}{n_1+n_2}$$

### Exercise 1.2
In blank the cell below, implement a function `class_priors(training_data)` that takes a dictionary of training data  and returns a dictionary that maps the name of each class to the class prior for that class.

Once you have done this, test it out on the training data above.

In [135]:
def class_priors(training_data):
    docCount = sum(len(docs) for docs in training_data.values())
    return {category: len(docs) / docCount for category, docs in training_data.items()}
    
trainingData = {"weather": weather_sents_train, "football": football_sents_train}
print(class_priors(trainingData))
        

{'weather': 0.6, 'football': 0.4}


### Conditional probabilities
We now turn to the problem of how to calculate (estimates of) the probabilities such as $P(\,d\,|\,\mbox{weather}\,)$ and $P(\,d\,|\,\mbox{football}\,)$, for some document $d$. The problem we have is that $d$ is a document, potentially a long document, that we won't have seen in the training data.

To address this, the Naïve Bayes model of document classification makes a major simplifying assumption.  In particular, it is assumed that the probabiity that different words occur in a document are independent of one another.

For example, if $d=\mbox{"today is looking cloudy"}$ then this assumption tells us that:

\begin{eqnarray*}
P(\,\mbox{"today is looking cloudy"}\,|\,\mbox{weather}\,) &=& P(\{\mbox{"today"},\mbox{"is"},\mbox{"looking"},\mbox{"cloudy"}\}\,|\,\mbox{weather}\,)\\
&=& P(\,\mbox{"today"}\,|\,\mbox{weather}\,)\times P(\mbox{"is"}\,|\,\mbox{weather}\,)\times P(\mbox{"looking"}\,|\,\mbox{weather}\,)\times P(\mbox{"cloudy"}\,|\,\mbox{weather}\,)
\end{eqnarray*}

For the general case, with class $c$ and document $d=\{w_1,\ldots,w_n\}$, we have:
\begin{eqnarray*}
P(\,d\,|\,c\,) &=& P(\,\{w_1,\ldots,w_n\}\,|\,c\,)\\
&=& \prod_{i=1}^n P(\,w_i\,|\,c\,)
\end{eqnarray*}

The point is that it is plausible that given a reasonable amount of training data, we can make reasonable estimates of the probabilities $P(\,\mbox{"today"}\,|\,\mbox{weather}\,)$, $P(\mbox{"is"}\,|\,\mbox{weather}\,)$, $P(\mbox{"looking"}\,|\,\mbox{weather}\,)$, and  $P(\mbox{"cloudy"}\,|\,\mbox{weather}\,)$. 

We now look at how that is done.

### Estimating conditional probabilities
So we have established that we need estimates of probabilities such as: $P(\,\mbox{"cloudy"}\,|\,\mbox{weather}\,)$, $P(\mbox{"is"}\,|\,\mbox{weather}\,)$, $P(\mbox{"today"}\,|\,\mbox{weather}\,)$, and $P(\mbox{"looking"}\,|\,\mbox{weather}\,)$.

How can these probabilities be estimated from the training data?

Look at the training data we set up above. There are 3 documents of class `weather` and within these documents there are a total of 11 tokens (8 different types).

From the data we can estimate the following probabilities:
- the probability of seeing "today" in a `weather` document is $\frac{2}{11}$,  
i.e. $P(\mbox{"today"}\,|\,\mbox{weather})=\frac{2}{11}$;
- the probability of seeing "it" in a `weather` document is $\frac{2}{11}$,  
i.e. $P(\mbox{"it"}\,|\,\mbox{weather})=\frac{2}{11}$;
- the probability of seeing "is" in a `weather` document is $\frac{2}{11}$,  
i.e. $P(\mbox{"is"}\,|\,\mbox{weather})=\frac{2}{11}$;
- the probability of seeing "raining" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"raining"}\,|\,\mbox{weather})=\frac{1}{11}$;
- the probability of seeing "looking" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"looking"}\,|\,\mbox{weather})=\frac{1}{11}$;
- the probability of seeing "cloudy" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"cloudy"}\,|\,\mbox{weather})=\frac{1}{11}$;
- the probability of seeing "nice" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"nice"}\,|\,\mbox{weather})=\frac{1}{11}$; and
- the probability of seeing "weather" in a `weather` document is $\frac{1}{11}$,  
i.e. $P(\mbox{"weather"}\,|\,\mbox{weather})=\frac{1}{11}$
- the probability of seeing any other word in a `weather` document is 0.
Notice that all of these conditional probabilities sum to $1$.

We can do the same thing for the `football` documents:
- the probability of seeing "city" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"city"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "looking" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"looking"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "good" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"good"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "advantage" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"advantage"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing "united" in a `football` document is $\frac{1}{5}$,  
i.e. $P(\mbox{"united"}\,|\,\mbox{weather})=\frac{1}{5}$;
- the probability of seeing any other word in a `football` document is 0.


### Exercise 1.3
We now look at how to implement the calculation of conditional probabilties.

In the empty cell below, define a function `cond_probs(training_data)` that takes training data and returns a dictionary that maps the name of a class, $c$, onto a dictionary that maps each word, $w$ to the conditional probability for that word given that class, i.e. $P(w|c)$.



In [143]:
def cond_probs(training_data):
    condDict = {}
    for c, docs in training_data.items():
        totalTokenCount = sum(len(doc.split()) for doc in docs)
        tokenDicts = listTokenDicts(docs, c)
        docTokenCount = {}
        for tokenDict, name in tokenDicts:
            for token in tokenDict:
                docTokenCount.setdefault(token, 0)
                docTokenCount[token] += 1
        condDictClass = {token: count / totalTokenCount for token, count in docTokenCount.items()}
        condDict[c] = condDictClass
    return condDict

print(cond_probs(trainingData))

{'weather': {'today': 0.18181818181818182, 'it': 0.18181818181818182, 'is': 0.18181818181818182, 'raining': 0.09090909090909091, 'looking': 0.09090909090909091, 'cloudy': 0.09090909090909091, 'nice': 0.09090909090909091, 'weather': 0.09090909090909091}, 'football': {'city': 0.2, 'looking': 0.2, 'good': 0.2, 'advantage': 0.2, 'united': 0.2}}


### Computing the conditional probability of a document

We have created implementations of the following functions:
- `class_priors(training_data)` that computes estimates of the class priors from training data;
- `cond_probs(training_data)` that computes estimates of the conditional probability of a word given a class from training data

Let us suppose that we have applied these functions to our training data as follows.

```
c_priors = class_priors(train_data)
c_probs = cond_probs(train_data)
```

`c_priors` and `c_probs` define the classifier.

### Exercise 1.4
In the cell below, complete the function `classify(doc,c_priors,c_probs)`. It should return the class that the classifier assigns to the document, `doc`. 

In the event of a tie, the function should randomly chose one of the classes (see `random.choice`). 

- Write your function in a way that allows for the possibilty of any number of classes.
- Assume that the document, `doc`, is represented as a dictionary that maps words (in the document) to `True`.

In [144]:
import random

def classify(doc,priors,c_probs):
    classProb = {}
    for c in priors:
        prodCondProb = 1
        for token, b in doc: prodCondProb *= c_probs[c].get(token, 0)
        classProb[c] = prodCondProb * priors[c]
    maxProb = 0
    mostLikelyClasses = []
    for c in classProb:
        if classProb[c] > maxProb:
            maxProb = classProb[c]
            mostLikelyClasses = [c]
        elif classProb[c] == maxProb:
            mostLikelyClasses.append(c)
    return random.choice(mostLikelyClasses)
        
        
   
        
c_priors = class_priors(trainingData)
c_probs = cond_probs(trainingData)
sent = "looking cloudy today"
doc = {(word, True) for word in sent.split()}
classify(doc,c_priors,c_probs)

'weather'

### A problem
There is a problem with the classifier that we have written. 

### Exercise 2.1
* Can you find example sentences which give you different answers on different runs of the classifier
* Can you explain why?

In [47]:
'''
Sentences that contain words that aren't in the training data for a given class will yield 0 for the probability of said word
given said class and hence an overall probability of 0 for that class. As a result, a sentence which contains a word in no
training sets for all classes will yield a probability of 0 for every class, and hence the classifier will choose randomly 
between them
'''

"\nSentences that contain words that aren't in the training data for a given class will yield 0 for the probability of said word\ngiven said class and hence an overall probability of 0 for that class. As a result, a sentence which contains a word in no\ntraining sets for all classes will yield a probability of 0 for every class, and hence the classifier will choose randomly \nbetween them\n"

### Add one smoothing
It will often be the case that a word appears in documents of one class, but not in any documents of another class.
- For example, consider the document "city looking cloudy today".
- While "city" appears in documents of class `football`, it does not appear in any documents of class `weather`. 
- Thus, the conditional probabiity $P(\,\mbox{city}\,|\,\mbox{weather}\,)$ is equal to zero.
- We are **multiplying** probabilities, so the document ends up with a score of zero even though all of the other words in the document suggest that it is of class `weather`.

To get around this we need to do something called **smoothing** in order to avoid zero probabilities. 

In particular, we will implement a version of smoothing called **add-one smoothing**. This involves adding a count of one to all of the known vocabulary.

### Known vocabulary
The words that appear in documents in the training data are collectively described as the known vocabulary. These are the words that the classifier can learn something about. If the classifier is asked to classify a document that contains any words that are not in the known vocabulary then the classifier will simply ignore them.

### Exercise 2.2
In the cell below, write a function `known_vocabulary(training_data)` that takes some training and returns a set containing all of words that appear in documents in the training data.

In [145]:
def known_vocabulary(training_data):
    return {token for docs in training_data.values() for doc in docs for token in doc.split()}

print(known_vocabulary(trainingData))

{'raining', 'nice', 'city', 'looking', 'united', 'is', 'cloudy', 'today', 'it', 'advantage', 'good', 'weather'}


### Implementing add-one smoothing
As the name suggests, add-one smoothing involves adding counts.

In particular, for each word, $w$, in the known vocabulary and each class, $c$, we add one extra count to our record of how many times $w$ appears in documents of class $c$. We are, in effect, hallucinating counts. The reason for doing this is that it means that we avoid zero probabilties.

### Exercise 2.3
In the blank cell below copy in your code for the `cond_probs` function that you wrote earlier. Then adapt this code so that it implements the add-one smoothing scheme.
- You will find it useful to use your `known_vocabulary` function.
- If there are $k$ words in your known vocabulary, then you will add $k$ counts for each class. 
- Therefore, when calculating conditional probabilities, you need to add $k$ to the denominator to account for these extra counts.

Test out your code.

In [146]:
def cond_probs(training_data):
    condDict = {}
    knownVocabulary = known_vocabulary(training_data)
    for c, docs in training_data.items():
        totalTokenCount = sum(len(doc.split()) for doc in docs)
        tokenDicts = listTokenDicts(docs, c)
        docTokenCount = {}
        for tokenDict, name in tokenDicts:
            for token in tokenDict:
                docTokenCount.setdefault(token, 0)
                docTokenCount[token] += 1
        for token in knownVocabulary:
            docTokenCount.setdefault(token, 0)
            docTokenCount[token] += 1
        condDictClass = {token: count / (totalTokenCount + len(knownVocabulary)) for token, count in docTokenCount.items()}
        condDict[c] = condDictClass
    return condDict

print(cond_probs(trainingData))

{'weather': {'today': 0.13043478260869565, 'it': 0.13043478260869565, 'is': 0.13043478260869565, 'raining': 0.08695652173913043, 'looking': 0.08695652173913043, 'cloudy': 0.08695652173913043, 'nice': 0.08695652173913043, 'weather': 0.08695652173913043, 'city': 0.043478260869565216, 'united': 0.043478260869565216, 'advantage': 0.043478260869565216, 'good': 0.043478260869565216}, 'football': {'city': 0.11764705882352941, 'looking': 0.11764705882352941, 'good': 0.11764705882352941, 'advantage': 0.11764705882352941, 'united': 0.11764705882352941, 'raining': 0.058823529411764705, 'nice': 0.058823529411764705, 'is': 0.058823529411764705, 'cloudy': 0.058823529411764705, 'today': 0.058823529411764705, 'it': 0.058823529411764705, 'weather': 0.058823529411764705}}


### Smoothing class priors
It is possible (though perhaps unlikely) that the training data does not contain any data for one class. In order to take care of this, we also smooth the class priors.

### Exercise 2.4
In the cell below, revise the `class_priors` function so that it adds one to the count of each class.

Check that the sum of the priors for each of the classes is $1$.

In [147]:
def class_priors(training_data):
    docCount = sum(len(docs) + 1 for docs in training_data.values())
    return {category: (len(docs) + 1) / docCount for category, docs in training_data.items()}
    
trainingData = {"weather": weather_sents_train, "football": football_sents_train}
print(class_priors(trainingData))

{'weather': 0.5714285714285714, 'football': 0.42857142857142855}


### Ignoring OOV words
We now look at how to update the `classify` function that we wrote earlier so that it ignores out of vocabulary words that appear in a document being classified.

### Exercise 2.5
In the blank cell below, copy the `classify` method you wrote earlier and update it so that words not in the known vocabulary are ignored.
- You will want to add an additional argument to the `classify` function that is a set containing the known vocabulary.

In [148]:
import random

def classify(doc,priors,c_probs, knownVocabulary):
    classProb = {}
    knownTokens = [token for token, b in doc if token in knownVocabulary]
    for c in priors:
        prodCondProb = 1
        for token in knownTokens: prodCondProb *= c_probs[c][token]
        classProb[c] = prodCondProb * priors[c]
    maxProb = 0
    mostLikelyClasses = []
    for c in classProb:
        if classProb[c] > maxProb:
            maxProb = classProb[c]
            mostLikelyClasses = [c]
        elif classProb[c] == maxProb:
            mostLikelyClasses.append(c)
    return random.choice(mostLikelyClasses)
        
c_priors = class_priors(trainingData)
c_probs = cond_probs(trainingData)
sent = "looking cloudy today a"
doc = {(word, True) for word in sent.split()}
classify(doc,c_priors,c_probs, known_vocabulary(trainingData))

'weather'

### Underflow

We need to address one final problem concerning the multiplication of probabilities.

Recall this equation from earlier:

\begin{eqnarray*}
P(\,d\,|\,c\,) &=& P(\,\{w_1,\ldots,w_n\}\,|\,c\,)\\
&=& \prod_{i=1}^n P(\,w_i\,|\,c\,)
\end{eqnarray*}

This tells us that in order to compute $P(\,d\,|\,c\,)$ for some document $d$ and class $c$, we must multiply $n$ conditional probabilities, one for each word in the document.

While in our toy example, this is not an issue. However, in a more realistic settings, where we had thousands of documents, each of which contained multiple paragraphs, we would find ourselves multiplying large numbers of very small probabilities. This would lead to **underflow**.

To avoid ths problem, we will add the log of probabilties. 

To understand why this is a reasonable thing to do let us recall a comparison from earlier:

$$P(\,d\,|\,\mbox{weather}\,)\cdot P(\,\mbox{weather}\,) \qquad\qquad \mbox{versus} \qquad\qquad P(\,d\,|\,\mbox{football}\,)\cdot P(\,\mbox{football}\,)$$

Our goal is to determine which of the values (on the left and right) is larger (or determine that they are equal). 

It should be clear that we will get exactly the same answer to this question by making the following comparsion.

$$\log(P(\,d\,|\,\mbox{weather}\,)) + \log(P(\,\mbox{weather}\,)) \qquad\qquad \mbox{versus} \qquad\qquad \log(P(\,d\,|\,\mbox{football}\,)) + \log(P(\,\mbox{football}\,))$$

Thus, rather than calculating conditional probabilities as described above, we will calculate log conditional probabilities like this:

\begin{eqnarray*}
\log(P(\,\mbox{"today is looking cloudy"}\,|\,\mbox{weather}\,)) &=& \log(P(\{\mbox{"today"},\mbox{"is"},\mbox{"looking"},\mbox{"cloudy"}\}\,|\,\mbox{weather}\,))\\
&=& \log(P(\,\mbox{"today"}\,|\,\mbox{weather}\,))\ +\\
&&\log(P(\mbox{"is"}\,|\,\mbox{weather}\,))\ +\\
&&\log(P(\mbox{"looking"}\,|\,\mbox{weather}\,))\ + \\
&&\log(P(\mbox{"cloudy"}\,|\,\mbox{weather}\,))
\end{eqnarray*}

For the general case, with class $c$ and document $d=\{w_1,\ldots,w_n\}$, we have:

\begin{eqnarray*}
\log(P(\,d\,|\,c\,)) &=& \log(P(\,\{w_1,\ldots,w_n\}\,|\,c\,))\\
&=& \sum_{i=1}^n \log(P(\,w_i\,|\,c\,))
\end{eqnarray*}


### Exercise 2.6
In the blank cell below, make a copy of the cell containing the definition of `classify`.

Adapt the code so that it adds logs of probabilties rather than multiplies probabilities.

In [149]:
import random
from math import log, inf

def classify(doc,priors,c_probs, knownVocabulary):
    classProb = {}
    knownTokens = [token for token, b in doc if token in knownVocabulary]
    for c in priors:
        sumCondProb = sum(log(c_probs[c][token]) for token in knownTokens)
        classProb[c] = sumCondProb + log(priors[c])
    maxProb = -inf
    mostLikelyClasses = []
    for c in classProb:
        if classProb[c] > maxProb:
            maxProb = classProb[c]
            mostLikelyClasses = [c]
        elif classProb[c] == maxProb:
            mostLikelyClasses.append(c)
    return random.choice(mostLikelyClasses)
        
c_priors = class_priors(trainingData)
c_probs = cond_probs(trainingData)
sent = "looking cloudy today a"
doc = {(word, True) for word in sent.split()}
classify(doc,c_priors,c_probs, known_vocabulary(trainingData))

'weather'

### Constructing a Naïve Bayes classifier

We are now almost ready to run our Naïve Bayes classifier on a set of test data. 

In [150]:
weather_sents_test = [
    "the weather today is nice",
    "it is raining cats and dogs",
    "the weather here is wet",
    "it was hot today",
    "rain due tomorrow",
]

football_sents_test = [
    "what a great goal that was",
    "poor defending by the city center back",
    "wow he missed a sitter",
    "united are a shambles",
    "shots raining down on the keeper",
]

weather_data_test = [({(word, True) for word in sent.split()}, "weather") for sent in weather_sents_test] 
football_data_test = [({(word, True) for word in sent.split()}, "football") for sent in football_sents_test]
test_data = weather_data_test + football_data_test



### Exercise 2.7
* Complete the NBClassifier `class` which can be used to carry out classification.  Add your code (from earlier) to the methods `set_known_vocabulary`, `set_priors` and `set_cond_probs` and `classify`.  
* Run your NB classifier on both the training data and the test data.

In [151]:
class NBClassifier(ClassifierI):
    
    def __init__(self):
        self._labels=["P","N"]
        pass
    
    def labels(self):
        return self._labels
    
    def _set_known_vocabulary(self,training_data):
         self.known = {token for docs in training_data.values() for doc in docs for token in doc.split()}
       
    def _set_priors(self,training_data):
        docCount = sum(len(docs) + 1 for docs in training_data.values())
        self.priors = {category: (len(docs) + 1) / docCount for category, docs in training_data.items()}
      
    def _set_cond_probs(self,training_data):       
        condDict = {}
        knownVocabulary = self.known
        for c, docs in training_data.items():
            totalTokenCount = sum(len(doc.split()) for doc in docs)
            tokenDicts = listTokenDicts(docs, c)
            docTokenCount = {}
            for tokenDict, name in tokenDicts:
                for token in tokenDict:
                    docTokenCount.setdefault(token, 0)
                    docTokenCount[token] += 1
            for token in knownVocabulary:
                docTokenCount.setdefault(token, 0)
                docTokenCount[token] += 1
            condDictClass = {token: count / (totalTokenCount + len(knownVocabulary)) for token, count in docTokenCount.items()}
            condDict[c] = condDictClass
        self.conds = condDict
        
    
    def train(self,training_data):
        self._set_known_vocabulary(training_data)
        self._set_priors(training_data)
        self._set_cond_probs(training_data)
    
    def classify(self,doc):
        classProb = {}
        strippedDoc = doc[0]
        knownTokens = [token for token, b in strippedDoc if token in self.known]
        for c in self.priors:
            sumCondProb = sum(log(c_probs[c][token]) for token in knownTokens)
            classProb[c] = sumCondProb + log(self.priors[c])
        maxProb = -inf
        mostLikelyClasses = []
        for c in classProb:
            if classProb[c] > maxProb:
                maxProb = classProb[c]
                mostLikelyClasses = [c]
            elif classProb[c] == maxProb:
                mostLikelyClasses.append(c)
        return random.choice(mostLikelyClasses)
        
    #the parent class classifierI has a classifier_many class which applies self.classify to every item in a list
    #so it does not need to be defined here.
    #https://www.nltk.org/_modules/nltk/classify/api.html
    
    

In [152]:
#create an instance of an NBClassifier, train it and call self.classify_many on the test data
classifier = NBClassifier()
classifier.train(trainingData)
classifier.classify_many(test_data)

['weather',
 'weather',
 'weather',
 'weather',
 'weather',
 'weather',
 'football',
 'weather',
 'football',
 'weather']

['weather',
 'weather',
 'weather',
 'weather',
 'weather',
 'weather',
 'football',
 'weather',
 'football',
 'weather']